In [1]:
from selenium import webdriver as wd
import urllib

In [2]:
target_url = 'http://www.opinet.co.kr/searRgSelect.do'

In [3]:
driver = wd.Chrome('../tool/chromedriver.exe')

In [4]:
# 해당 사이트가 중간 페이지를 직접 접속하는 부분을 거부하기 때문에
import time
for n in range(2) :
    driver.get(target_url)
    time.sleep(5)

1. 시도 정보를 추출하여 리스트화
2. 리스트를 반복하면서 해당 시도 밑에 시군구 리스트화
3. 시군구 리스트를 반복하면서 읍면동 리스트화
4. 읍면동 리스트 반복하면서 조회 -> 다운로드
5. 사이트 관찰 결과, 시도, 시군구, 읍면동 정보가 새로 세팅되면 화면의 DOM이 모두 새로 구성되는 문제가 있어서 반복문 안에서는 반드시 시도, 시군구, 읍면동을 새로 잡아줘야 한다!(매우 중요)
6. (여기서는) 시군구를 반복하면서 바로 조회 및 다운로드로 전개하겠다

In [5]:
# 시도 리스트 획득
sido = driver.find_element_by_id('SIDO_NM0')
sidos = [option.get_attribute('value')
         for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
         if option.get_attribute('value')]
# 데이터의 상위값, 하위값 조사를 통해 샘플링으로 확인
sidos[:2], sidos[-2:]

(['서울특별시', '부산광역시'], ['경상남도', '제주특별자치도'])

In [7]:
# 2. 시도 리스트를 반복하면서 해당 시도 밑에 시군구 리스트화
# sidos 중에 '부산광역시' 대상으로 수집하겠다
for sido_name in sidos[1:2] :
    # 2-1. sido 대상 찾아서
    sido = driver.find_element_by_id('SIDO_NM0')
    # 2-2. sido 값을 설정
    sido.send_keys(sido_name)
    # 2-2-2. 잠시대기(ajax 통신 후 화면이 재구성될 때까지)
    time.sleep(4)
    # 2-3. 시군구 리스트 획득
    sigungus = [option.get_attribute('value')
         for option in driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')
         if option.get_attribute('value')]
    print(sido_name, sigungus[:2])
    # 2-4. 반복~ 3단계로 진입
    # 3. 시군구 리스트를 반복하면서 -> 잠시대기(5초대기) -> 엑셀저장
    for sigungu_name in sigungus :
        # 3-1. 시군구 태그를 찾아서
        sigungu = driver.find_element_by_id('SIGUNGU_NM0')
        # 3-2. 시군구값 설정
        sigungu.send_keys(sigungu_name)
        # 3-3. 잠시 대기
        time.sleep(5)
        # 3-4. 엑셀 저장
        driver.find_element_by_id('glopopd_excel').click()

부산광역시 ['강서구', '금정구']


### 데이터 수집 완료

- 수십 개의 엑셀 파일을 반복적으로 읽어서 DataFrame으로 구성하기

In [9]:
# glob 특정 위치상에 존재하는 파일들을 패턴을 지정하여 목록을 구할 수 있다
from glob import glob
import pandas as pd

In [10]:
# 부산 관할 시군구에 조회된 엑셀 자료를 읽을 수 있게 파일 경로를 리스트화
down_files = glob('C:/Users/User/Downloads/지역*.xls')
down_files

['C:/Users/User/Downloads\\지역_위치별(주유소) (1).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (10).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (11).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (12).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (13).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (14).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (15).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (2).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (3).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (4).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (5).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (6).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (7).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (8).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소) (9).xls',
 'C:/Users/User/Downloads\\지역_위치별(주유소).xls']

In [ ]:
# conda install xlrd

In [12]:
tmp_raws = []
for file in down_files :
    tmp = pd.read_excel(file, header = 2)
    tmp_raws.append(tmp)
# 데이터 확인
st_raw = pd.concat(tmp_raws)

In [13]:
st_raw

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,부산광역시,청룡명품주유소,부산 금정구 중앙대로2130,현대오일뱅크,051-508-5150,N,-,1442,1292,-
1,부산광역시,훼미리알뜰주유소,부산 금정구 두실로 2 (남산동),알뜰주유소,051-582-5882,N,-,1443,1293,-
2,부산광역시,에스원에너지(주)금강주유소,부산 금정구 중앙대로 1972,SK에너지,051-513-2700,N,-,1443,1293,-
3,부산광역시,은마석유 노포주유소,부산 금정구 중앙대로 2191 (노포동),GS칼텍스,051-508-1645,Y,-,1443,1293,1050
4,부산광역시,광신석유(주)직영경부고속주유소,부산 금정구 중앙대로1720번길 7 (부곡동),SK에너지,051-517-6011,Y,-,1455,1315,1010
5,부산광역시,지에스칼텍스(주)신창주유소,부산 금정구 중앙대로 1698 (부곡동),GS칼텍스,051-581-1212,N,1730,1455,1315,990
6,부산광역시,A1 주유소,부산 금정구 반송로 364 (서동),현대오일뱅크,051-532-5157,Y,-,1458,1297,-
7,부산광역시,현대오일뱅크㈜ 직영 금정셀프주유소,부산 금정구 중앙대로 1664 (부곡동),현대오일뱅크,051-583-8951,Y,-,1463,1323,-
8,부산광역시,거북이주유소(SK),부산 금정구 중앙대로 1672 (부곡동),SK에너지,051-516-5151,Y,-,1463,1323,-
9,부산광역시,현대오일뱅크㈜직영 부곡셀프주유소,부산 금정구 중앙대로 1597 (부곡동),현대오일뱅크,051-516-2888,Y,1812,1463,1323,-
